# Latihan-6 Half Space in 1D (Angle, Thickness, Fluid)

Latihan ini untuk pemodelan half space atau disebut juga blocky model, atau interface model yang terdiri dari 2-3 block layers dengan variasi ketebalan untuk memodelkan respon amplitudo terhadap interface antara layers
<br><br> _citation for this page: "Adi Widyantoro, 2021, Pertamina Seismic QI Course, Latihan-6 Halfspace in 1D - Angle Thickness Fluid.ipynb, accessed MM DD, YYYY."_  
<br>
>(update terakhir tanggal 1 Juli 2021 oleh Adi Widyantoro)
<hr>

__Tahap-1__ 
<br> Program setup dan display sama seperti latihan sebelumnya. Gunakan hasil dari Latihan-2 data untuk latihan selanjutnya
<br> 

        special notes_: brugeslibrary adalah kumpulan pre-defined functions oleh Agile Geoscience bruges:
        https://github.com/agile-geoscience/bruges
        
        the library's credits go to bruges contributors: Evan Bianco, Ben Bougher, Matt Hall, Alessandro Amato del Monte, 
        Wes Hamlyn, Sean Ross-Ross

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'png'
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd
import math
from scipy.signal import hilbert
from f07reflect import fullzoep, ei_norm
from f07gassmann import fluidsub

__Tahap-2__ 
<br> Mendapatkan nilai rata-rata untuk masing-masing interval. Data dari Latihan-2 sebelumnya adalah sbb:

In [ ]:
oil_mean = np.array([3.260,1.860,2.29])
ssd_mean = np.array([3.440,1.970,2.37])
sh_mean = np.array([3.310,1.730,2.51])
brine_mean = np.array([3.500,2.050,2.35])
coal_mean = np.array([2.450,1.660,1.37])

__Tahap-3__ 
<br> Model konseptual thickness divisualisasikan dalam bentuk sample numbers (e.g. asumsi 1 sampel unit = 1 unit kedalaman) 

In [ ]:
"""
Visualisasi ini tidak terlalu straightforward, memerlukan workaround dan akan diganti dengan yang lebih baik
tetapi untuk saat ini, thickness dimodelkan dengan memanipulasi jumlah sampel = 500 unit
lapisan pertama-kedua-ketiga adalah fraksi total jumlah sampel yang akan dipartisi sesuai dengan input fraksi nya

misalnya fraksi lapisan pertama = 0.4, maka thickness nya menjadi 0.4 * 500 = 200 unit thickness
jika fraksi lapisan kedua = 0.6, maka thickness lapisan kedua menjadi (0.6-0.4)*500 = 100 unit thickness
contoh lain jika lapisan kedua adalah thin layer = 0.42, maka thickness lapisan kedua (0.42-0.4)*500 = 10 unit
untuk lapisan ketiga tidak perlu ditentukan karena akan mengambil sisanya
"""
n_samples = 500
interface1 = int(0.4*n_samples) # mis: 500*0.4 = 200, atau 200 unit pertama menjadi interface-1
interface2 = int(0.6*n_samples) # mis: 500*0.6 = 300, atau 100 unit menjadi interface-2

In [ ]:
"""
default wavelet yang digunakan adalah Ricker central frequency 20Hz, 200samples
eksplorasi pilihan rotasi wavelet dengan mengubah ph=0 menjadi rotasi yang diinginkan misalnya ph=-90deg
untuk apreasiasi visualisasi bandlimited amplitude responses  
"""
f, l, dt, ph = 20, 200, 0.4, 0
t = np.linspace(-l/2, (l-dt)/2, int(l/dt))/1000
zerophase = (1.0 - 2.0*(np.pi**2)*(f**2)*(t**2)) * np.exp(-(np.pi**2)*(f**2)*(t**2))
h = hilbert (zerophase)
theta = ph*np.pi/180
wavelet = np.cos(theta)*h.real-np.sin(theta)*h.imag


#----------pilihan rentang min/max theta (angles.deg)--------------
thetamin=0; thetamax=50
ang = np.arange(thetamin,thetamax+1,2)
z = np.arange(n_samples)

__Tahap-4__ 
<br> variasi fluida dengan konvolusi elastic impedance (angle dependence) terhadap wavelet


_credit: codes are inspired by original aadm's program: //geophysical_notes_master/relationship-reflectivity-elastic-impedance_Simm-Bacon.ipynb_

### lapisan BRINE

In [ ]:
"-----------------INPUT PROPERTIES-------------------------------------"

#------------Lapisan pertama dan ketiga--------------------------------
vpsh=sh_mean[0]; vssh=sh_mean[1]; rhosh=sh_mean[2]
aish=sh_mean[0]*sh_mean[2]
norm = vpsh,vpsh,vpsh

#------------Lapisan kedua---------------------------------------------
vpbr=brine_mean[0]; vsbr=brine_mean[1]; rhobr=brine_mean[2]
aibr=round((vpbr*rhobr),2)
#----------------------------------------------------------------------

In [ ]:
"""
model yang digunakan saat ini masih zero reflectivity untuk setiap angle
dan bukan full gathers dari reflectivity equation. 
model ini akan diganti menjadi full gathers dalam waktu dekat, tetapi
untuk saat ini cukup memberikan ilustrasi efek thickness, wavelet dan fluid
"""
#-------------layer model----------------------------------------------
modelbr=np.zeros((n_samples,4))
modelbr[:interface1,:]=[vpsh,vssh,rhosh, aish]
modelbr[interface2:,:]=[vpsh,vssh,rhosh, aish]
modelbr[interface1:interface2,:]=[vpbr,vsbr,rhobr,aibr]
aibr=modelbr[:,3]
#elastic impedance
eibr, rcbr, synbr = (np.zeros((n_samples,ang.size)) for _ in range(3))
for i,alpha in enumerate(ang):
    eibr[:,i] = ei_norm(modelbr[:,0], modelbr[:,1], modelbr[:,2], alpha, norm)
    rcbr = (eibr[1:,i] - eibr[:-1,i]) / (eibr[1:,i] + eibr[:-1,i])
    rcbr = np.append(np.nan, rcbr)
    rcbr = np.nan_to_num(rcbr)
    synbr[:,i] = np.convolve(rcbr, wavelet, mode='same')

In [ ]:
print(vpbr, vsbr, rhobr, aibr[1], sep=" | ")

### lapisan HIDROKARBON

In [ ]:
"-----------------INPUT PROPERTIES-------------------------------------"
"""
jika menggunakan pilihan gassmann replacement, eksplorasi opsi variasi fluid mixture
dengan mengubah f2mix dengan rentang 0.0 - 1.0 untuk melihat efeknya terhadap model
eksplorasi opsi juga dapat dilakukan dengan mengganti mixture model dari isostress Reuss
menjadi lebih patchy dengan Brie atau Voigt seperti yang telah dibahas pada Latihan-3
"""
#--------Lapisan pertama dan ketiga sama dengan brine di atas----------

#-----------------Lapisan kedua (dari data sumur)----------------------
#vphc=oil_mean[0]; vshc=oil_mean[1]; rhohc=oil_mean[2]

#--------pilihan gassmann (use ctrl+ / for multiple hastags)-----------
vp1=brine_mean[0]; vs1=brine_mean[1]; rho1=brine_mean[2]; 
ai1=brine_mean[0]*brine_mean[2]

vsh=0.195; phi=0.207; rho_br=1.09; rho_oil=0.78; k_qz=36.6; k_sh=20.9  # dari latihan-3
kmin = 0.5*((vsh*k_sh+(1-vsh)*k_qz)+(vsh/k_sh)+(1-vsh/k_qz))
k_br=2.8; k_oil=0.94; k_gas=0.06 
f2mix=0 # fluid mixture
#----------------------------------------------------------------------

In [ ]:
#-------------layer model----------------------------------------------
if f2mix==0:
    rhof1=rho_br; rhof2=rhof1
    kf1=k_br; kf2=kf1
    kfl2=kf1
    vphc=vp1; vshc=vs1; rhohc=rho1; aihc=ai1
else:
    f1mix=1-f2mix
    rhof1=rho_br; rhof2=rho_oil
    rhofl2=(f1mix*rhof1)+(f2mix*rhof2) # eff density
    kf1=k_br; kf2=k_oil
    kreuss=1/((f1mix/kf1)+(f2mix/kf2)) # eff modulus (Domenico, 1976)
    kvoigt=(f1mix*kf1)+(f2mix*kf2) # patchy approach (Mavko and Mukerji, 1998)
    kfl2=kreuss
    gassmn=fluidsub(vp1, vs1, rho1, phi, rhof1, rhofl2, kmin, kf1, kfl2)
    vphc=gassmn[0].round(2); vshc=gassmn[1].round(2); rhohc=gassmn[2].round(2)
    aihc=round((vphc*rhohc),2)
modelhc=np.zeros((n_samples,4))
modelhc[:interface1,:]=[vpsh,vssh,rhosh,aish]
modelhc[interface2:,:]=[vpsh,vssh,rhosh, aish]
modelhc[interface1:interface2,:]=[vphc,vshc,rhohc,aihc]
aihc=modelhc[:,3]

#elastic impedance
eihc, rchc, synhc = (np.zeros((n_samples,ang.size)) for _ in range(3))
for i,alpha in enumerate(ang):
    eihc[:,i] = ei_norm(modelhc[:,0], modelhc[:,1], modelhc[:,2], alpha, norm)
    rchc = (eihc[1:,i] - eihc[:-1,i]) / (eihc[1:,i] + eihc[:-1,i])
    rchc = np.append(np.nan, rchc)
    rchc = np.nan_to_num(rchc)
    synhc[:,i] = np.convolve(rchc, wavelet, mode='same')

In [ ]:
print(k_br, round(kfl2,4), sep=" | "); # QC jika Gassmann dan f2mix=0, maka seharusnya kfl2 = k_brine 

### Analisis dan iterasi perbandingan brine dan hidrokarbon layers dengan variasi angles

In [ ]:
gain=5
f=plt.subplots(figsize=(20, 5), sharey=True, facecolor='white')
mpl.style.use('default')
ax0 = plt.subplot2grid((1,15), (0,0), colspan=1)
ax1 = plt.subplot2grid((1,15), (0,1), colspan=3)
ax2 = plt.subplot2grid((1,15), (0,4), colspan=1)
ax3 = plt.subplot2grid((1,15), (0,5), colspan=3)
#------------brine--------------------------------------------
ax0.plot(aibr, z, 'k', lw=1); ax0.set_xlabel('AI Brine', size=16)
ax0.fill_betweenx(z, aibr, aibr.max(), facecolor='b')
for i in range(ang.size):
    tracebr=synbr[:,i]*gain
    ax1.plot(tracebr+i,z,color='k', linewidth=1)
    ax1.fill_betweenx(z,tracebr+i,i, where=tracebr+i>i, 
                      facecolor='darkblue', linewidth=0, alpha=0.6)
    ax1.fill_betweenx(z,tracebr+i,i, where=tracebr+i<i, 
                      facecolor='r', linewidth=0, alpha=0.6)
    ax1.set_xlim(-0.95,synbr.shape[1]-.1)
#-----------replaced with HC-----------------------------------
ax2.plot(aihc, z, 'k', lw=1); ax2.set_xlabel('AI HC', size=16)
ax2.fill_betweenx(z, aihc, aihc.max(), facecolor='g', alpha=0.4)
ax2.set_yticklabels([]); ax2.set_xlim(0.99*aihc.min(),1.01*aihc.max())    
for j in range(ang.size):
    tracehc = synhc[:,j]*gain
    ax3.plot(tracehc+j,z,color='k', linewidth=1)
    ax3.fill_betweenx(z,tracehc+j,j, where=tracehc+j>j, facecolor='darkblue', linewidth=0, alpha=0.6)
    ax3.fill_betweenx(z,tracehc+j,j, where=tracehc+j<j, facecolor='r', linewidth=0, alpha=0.6)   
    ax3.set_xlim(-0.95,synhc.shape[1]-.1)
ticks=int((thetamax/2))
for aa in [ax0, ax1, ax2, ax3]:
    aa.set_ylim(100,350)
    aa.invert_yaxis()
    aa.xaxis.set_ticks_position('top')
    aa.xaxis.set_label_position('top')
for bb in [ax1, ax3]:    
    bb.set_yticklabels([])
    bb.set_xticks((0,ticks/2,ticks))
    bb.set_xticklabels((0,ticks,thetamax))
for cc in [ax0, ax2]: 
    cc.grid(color='lightgrey', linestyle=':')
    cc.set_xlim(0.99*aihc.min(),1.01*aihc.max())

Prosedur di atas menggunakan pendekatan zero-offset reflectivity (EI2-EI1)/(E2+EI1) pada setiap angles. Bukan fully angle dependence reflectivities dengan relative amplitude dan frequency nya.<br>

tetapi dapat digunakan sebagai konsep untuk menganalisis variasi besaran amplitudo terhadap angle, terhadap thickness, general fluid content, dan wavelet phase<br>
<br>
Diskusikan dengan intruktur untuk mengubah: SUDUT DATANG, KETEBALAN, SATURASI, ROTASI FASA WAVELET yang akan berpengaruh terhadap variasi amplitudo
<br>

    _catatan_: wavelet interference terhadap thickness tidak dimodelkan dalam latihan ini

<h1><center>-Akhir Program Latihan-6-<center></h1>